#### (4月12日) Corpus Analysis

In [1]:
import numpy as np
import pandas as pd
import copy
import re
from tqdm import tqdm 

pd.options.mode.chained_assignment = None

In [2]:
# Unpickle
corpus_new = pd.read_pickle('savefiles/corpusfull_20220410.pkl')

In [3]:
def collapse(text):
    
    """
    Paragraphs extracted from XML files contain different numbers of sentences,
    and even incomplete sentences. This function collapses a list of paragraphs
    into a list of sentences or sentence-equivalents in advance of NLP processing.
    """
    
    doc_raw = ""
    punc = set([".", ",", ";", ":"])
    for i, para in enumerate(text):
        if para[-1] in punc:
            doc_raw += (para + " ")
        else:
            doc_raw += (para + ". ")
        
    return doc_raw

In [4]:
def preprocess_gen(text):
    
    """
    This function is primarily for removing references, and fixing spacing
    between/within sentences in preparation for scispaCy language modeling.
    """
    
    text = text.replace('\u200a', '').replace('\n', '')   # remove weird space code, newlines
    text = re.sub('\[(\d+)\]', '', text)                  # remove refs ([1], [23], etc.)
    text = text.replace(' ,', '').replace(' .', '.')      # fix spaces created by prev line
    text = text.replace(' ;', ';').replace(' :', ':')     # 
    text = text.replace('  ', ' ')
    
    return text

In [5]:
# Converts each doc from list of paras to one long string
for i in range(0,20):
    corpus_new.loc[i, "Text"] = collapse(corpus_new.loc[i, "Text"])

# Preprocess each doc before Spacy modeling
for i in range(0,20):
    corpus_new.loc[i, "Text"] = preprocess_gen(corpus_new.loc[i, "Text"])
    
# For practicing on single texts
# text = corpus_new.loc[0, "Text"]
# text

In [6]:
# Row indices of docs in respective corpora
jpen_i = [0,  1,  5, 10, 11, 12, 13, 14, 17, 19]
enen_i = [2,  3,  4,  6,  7,  8,  9, 15, 16, 18]

In [7]:
def initialize_results_df():
    
    """
    Results DF should be reset for every hypothesis tested.
    Note: Run BEFORE preprocess_tok to capture unanalyzed tokens.
    """
    
    results_df = corpus_new[['Group', 'Author', 'Title']]
    results_df['Title'] = pd.Series([title[:31] for title in results_df['Title']])  # prune title for readability
    
    # with word count
    for i in range(0,20):
        # Pull text from df
        text = corpus_new.loc[i, "Text"]
        # Split by word (" " for simplicity)
        word_ct = len(text.split(' '))
        # Put in results_df
        results_df.loc[i, 'Word Count'] = word_ct
    
    return results_df

### (4月23日) Search for tokens/strings

In [8]:
import scispacy
import spacy

nlp = spacy.load("en_core_sci_md")

In [9]:
# Get each text as scispaCy doc 
corpus_new['spacy_docs'] = [nlp(text) for text in corpus_new['Text']]

# Get each text as list of scispaCy sentences
sent_list = []
for i in range(0,20):
    s = list([x for x in corpus_new.spacy_docs[i].sents])
    sent_list.append(s)
corpus_new['spacy_sents'] = pd.Series(sent_list)
corpus_new

In [60]:
corpus_new

,Author,Year,Title,Text,ID,Group,spacy_docs,spacy_sents
0,Tamura,2012,Neural Network Development in Late Adolescents...,Emotional maturity and social awareness are im...,10.1371/journal.pone.0039527,JP-EN,"(Emotional, maturity, and, social, awareness, ...","[(Emotional, maturity, and, social, awareness,..."
1,Watanabe,2012,Diminished Medial Prefrontal Activity behind A...,Individuals with autism spectrum disorders (AS...,10.1371/journal.pone.0039561,JP-EN,"(Individuals, with, autism, spectrum, disorder...","[(Individuals, with, autism, spectrum, disorde..."
2,Sobhani,2012,Interpersonal Liking Modulates Motor-Related N...,Observing someone perform an action engages br...,10.1371/journal.pone.0046809,EN-EN,"(Observing, someone, perform, an, action, enga...","[(Observing, someone, perform, an, action, eng..."
3,Majdandžić,2012,The Human Factor: Behavioral and Neural Correl...,The extent to which people regard others as fu...,10.1371/journal.pone.0047698,EN-EN,"(The, extent, to, which, people, regard, other...","[(The, extent, to, which, people, regard, othe..."
4,Dixon,2012,The Decision to Engage Cognitive Control Is Dr...,Cognitive control is a fundamental skill refle...,10.1371/journal.pone.0051637,EN-EN,"(Cognitive, control, is, a, fundamental, skill...","[(Cognitive, control, is, a, fundamental, skil..."
5,Ohta,2013,Syntactic Computation in the Human Brain: The ...,Our goal of this study is to characterize the ...,10.1371/journal.pone.0056230,JP-EN,"(Our, goal, of, this, study, is, to, character...","[(Our, goal, of, this, study, is, to, characte..."
6,Deeley,2013,Using Hypnotic Suggestion to Model Loss of Con...,The feeling of voluntary control and awareness...,10.1371/journal.pone.0078324,EN-EN,"(The, feeling, of, voluntary, control, and, aw...","[(The, feeling, of, voluntary, control, and, a..."
7,Pawliczek,2013,Anger under Control: Neural Correlates of Frus...,Antisocial behavior and aggression are promine...,10.1371/journal.pone.0078503,EN-EN,"(Antisocial, behavior, and, aggression, are, p...","[(Antisocial, behavior, and, aggression, are, ..."
8,Jansma,2013,fMRI Guided rTMS Evidence for Reduced Left Pre...,Cognitive tasks that do not change the require...,10.1371/journal.pone.0080256,EN-EN,"(Cognitive, tasks, that, do, not, change, the,...","[(Cognitive, tasks, that, do, not, change, the..."
9,Lidzba,2013,Complex Visual Search in Children and Adolesce...,Complex visuospatial processing relies on dist...,10.1371/journal.pone.0085168,EN-EN,"(Complex, visuospatial, processing, relies, on...","[(Complex, visuospatial, processing, relies, o..."


In [72]:
from spacy.matcher import Matcher

doc = corpus_new['spacy_docs'][11]

pattern = [{'LEMMA': 'report'}]

matcher = Matcher(nlp.vocab)
matcher.add("REPORT", [pattern])
matches = matcher(doc)

# Iterate over the matches
for match_id, start, end in matches:
    # Get the matched span
    matched_span = doc[start:end]
    print('- ', matched_span.sent.text)
    
print(len(matches))

-  Although the graph-theoretical approaches are now being applied to studies on different aspects of the ASC brain (e.g., white matter connectivity and electrophysiological connectivity) –, only a few have applied graph-theoretical analysis to rs-fMRI data of this disorder –. Rudie et al. recently described disrupted local segregation and enhanced global integration in adolescents with ASC compared to neurotypical counterparts, while Barttfeld et al. have reported that changes in the small-worldness measure occurring at shifts between different cognitive states show opposite patterns in patients with ASC and controls.
-  None of the NCs reported any severe medical problem or history of any neurological or psychiatric problems.
-  Although we calculated all the above-mentioned global metrics, we reported mainly on r, C, L, and σ because some metrics bore a certain relationship (i.e., proportional or inverse) to other metrics.
-  Although almost all of the examined global metrics in adu

In [40]:
doc

Emotional maturity and social awareness are important for adolescents, particularly college students beginning to face the challenges and risks of the adult world. However, there has been relatively little research into personality maturation and psychological development during late adolescence and the neural changes underlying this development. We investigated the correlation between psychological properties (neuroticism, extraversion, anxiety, and depression) and age among late adolescents (n=25, from 18 years and 1 month to 22 years and 8 months). The results revealed that late adolescents became less neurotic, less anxious, less depressive and more extraverted as they aged. Participants then observed video clips depicting hand movements with and without a risk of harm (risk-taking or safe actions) during functional magnetic resonance imaging (fMRI). The results revealed that risk-taking actions elicited significantly stronger activation in the bilateral inferior parietal lobule, t

In [31]:
corpus_new.spacy_sents[0][0].lemma_

str

In [33]:
import re

matches = []

for i, sent in enumerate(corpus_new.spacy_sents[0]):
    if 'report' in sent.lemma_:
        #words = re.findall('report', sent.text)
        matches.append(tuple((i, sent.text)))
    
matches    

[(20,
  'For example, increases in white matter volume have been reported throughout childhood and adolescence, particularly in the prefrontal and parietal cortices (e.g.,.'),
 (21,
  'In addition, grey matter volume has been reported to increase in the prefrontal and parietal cortices during the preadolescent stage, followed by a steady decline during late adolescence.'),
 (25,
  'For example, a behavioral study using a mentalizing task requiring theory of mind and executive function reported that social abilities like ‘theory of mind’ continue to improve from adolescence to adulthood further suggesting that developmental changes continue throughout the late adolescent phase.'),
 (26,
  'Another study using a gambling task reported that the rate of risky choices did not significantly change between early (12–15 y.o.) and mid (15–18 y.o.) adolescence, but was significantly reduced in adulthood (25–35 y.o.).'),
 (28,
  'Functional neuroimaging studies of mental-state attribution have re

In [ ]:
def unique(ls):
    
    """
    Creates a list of unique items from an existing list.
    """
 
    unique_list = []
     
    for x in ls:
        if x not in unique_list:
            unique_list.append(x)
    
    return unique_list

In [ ]:
def preprocess_tok(doc):
    
    # Collect lemmas not tagged by spaCy as 1. punctuation, 2. digits, 3. URLs, or 4. stop words
    tokens = [tok.lemma_ for tok in doc if not (tok.is_punct | tok.is_digit | tok.like_url | tok.is_stop)]
    
    # Remove any tokens containing mid-string digits (e.g. "P5-a") or punc ('t(are')
    tokens = [tok for tok in tokens if not re.search("\d", tok)]
    tokens = [tok for tok in tokens if not re.search("\(", tok)]
    tokens = [tok for tok in tokens if not re.search("\)", tok)]
    
    # (4.13) Break apart hyphen- or slash-separated compounds
    seps = ['-', '–', '―',
            ';', ':',
            '\]', '\[', 
            '’', '”', 
            '>', '<', '/']
    for sep in seps:
        new_toks = []
        for tok in tokens:
            new_toks += tok.split(sep)
        tokens = new_toks
    
    # (4.13) Remove remaining abbreviations
    tokens = [tok for tok in tokens if not re.search("[a-zA-Z]\.[a-zA-Z]\.", tok)]
    tokens = [tok for tok in tokens if not re.search("\+", tok)]
    
    # Remove punc and small words (e.g. 'a', 'P', 'mm')
    punc_to_skip = set(['±', '=', '>', '<'])
    tokens = [tok for tok in tokens if tok not in punc_to_skip]     # can skip?
    tokens = [tok for tok in tokens if len(tok) > 3]    
       
    # Unify to lowercase (to simplify matching)
    tokens = [tok.lower() for tok in tokens]
    
    return tokens

In [ ]:
def find_tokens_unique_to_one_doc(i):
    
    other_docs = list(range(0,20))
    other_docs.remove(i)
    
    # Doc in question
    text_i = corpus_new.loc[i, "Text"]
    doc_i = nlp(text_i)
    tokens_i = unique(preprocess_tok(doc_i))
    
    # Iterate thru all 19 other docs
    for j in other_docs:
        text_j = corpus_new.loc[j, "Text"]
        doc_j = nlp(text_j)
        tokens_j = set(unique(preprocess_tok(doc_j)))
        
        for tok in tokens_i:
            if tok in tokens_j:
                tokens_i.remove(tok)
    
    return tokens_i            

In [ ]:
# Check performance of preprocessing function
toks_unique = find_tokens_unique_to_one_doc(5)
toks_unique

### (4.13) Comparing tokens not specific to one doc or corpus

In [ ]:
# Collect all tokens found in 2+ docs in JP-EN

tokens_shared_jpen = []

for i in tqdm(jpen_i):
    text_i = corpus_new.loc[i, "Text"]
    doc_i = nlp(text_i)
    tokens_i = preprocess_tok(doc_i)
    tokens_i_unique = find_tokens_unique_to_one_doc(i)
    tokens_shared_i = [tok for tok in tokens_i if tok not in set(tokens_i_unique)]
    for tok in tokens_shared_i:
        tokens_shared_jpen.append(tok)

tokens_shared_jpen_unique = unique(tokens_shared_jpen) # remove duplicates

print(f'Non-document-specific tokens in JP-EN: {len(tokens_shared_jpen_unique)}')

In [ ]:
# Collect all tokens found in 2+ docs in EN-EN

tokens_shared_enen = []

for i in tqdm(enen_i):
    text_i = corpus_new.loc[i, "Text"]
    doc_i = nlp(text_i)
    tokens_i = preprocess_tok(doc_i)
    tokens_i_unique = find_tokens_unique_to_one_doc(i)
    tokens_shared_i = [tok for tok in tokens_i if tok not in set(tokens_i_unique)]
    for tok in tokens_shared_i:
        tokens_shared_enen.append(tok)

tokens_shared_enen_unique = unique(tokens_shared_enen)
print(f'Non-document-specific tokens in EN-EN: {len(tokens_shared_enen_unique)}')

In [ ]:
# Find intersection of tokens appearing in ≧4 docs (≧2 in each corpus) 

toks_to_analyze = [tok for tok in tokens_shared_jpen_unique if tok in set(tokens_shared_enen_unique)]
print(f'{len(toks_to_analyze)} tokens to analyze')

### (4.13) Run overnight

In [ ]:
# Big Kahuna

from scipy.stats import ttest_ind

docs_df = initialize_results_df()
toks_df = pd.DataFrame()

for tok in tqdm(toks_to_analyze):
    for i in range(0,20):
        text = corpus_new.loc[i, "Text"]
        doc = nlp(text)
        toks_in_doc = preprocess_tok(doc)
        counter = 0
        for doctok in toks_in_doc:
            if str(doctok) == str(tok):
                counter += 1
        docs_df.loc[i, "count"] = counter
        docs_df.loc[i, "count_adj"] = (counter / docs_df.loc[i, 'Word Count']) * 1000  # per 1000 words
    
    # Separate values by group (JP-EN v. EN-EN)
    jp_ct    = list(docs_df[docs_df['Group'] == 'JP-EN'].loc[:, "count"])
    jp_ctadj = list(docs_df[docs_df['Group'] == 'JP-EN'].loc[:, "count_adj"])
    en_ct    = list(docs_df[docs_df['Group'] == 'EN-EN'].loc[:, "count"])
    en_ctadj = list(docs_df[docs_df['Group'] == 'EN-EN'].loc[:, "count_adj"])
    
    # Calculate means and run t-tests
    jp_ct_mean    = np.mean(jp_ct)
    jp_ctadj_mean = np.mean(jp_ctadj)
    en_ct_mean    = np.mean(en_ct)
    en_ctadj_mean = np.mean(en_ctadj)
        
    P_ct    = ttest_ind(jp_ct, en_ct).pvalue
    P_ctadj = ttest_ind(jp_ctadj, en_ctadj).pvalue
    
    # Add to DF
    entry = {'Token': tok,
             'JP-EN Mean Count': jp_ct_mean, 'JP-EN Mean Freq': jp_ctadj_mean,
             'EN-EN Mean Count': en_ct_mean, 'EN-EN Mean Freq': en_ctadj_mean,
             'P (count)': P_ct, 'P (freq)': P_ctadj}
    toks_df = toks_df.append(entry, ignore_index = True)    

In [ ]:
toks_df.to_pickle('savefiles/toksdf_20220413.pkl')

### Word-level analysis (4/18)

In [73]:
toks_df = pd.read_pickle('savefiles/toksdf_20220413.pkl')

In [76]:
toks_df[toks_df['Token'] == 'report']

,Token,JP-EN Mean Count,JP-EN Mean Freq,EN-EN Mean Count,EN-EN Mean Freq,P (count),P (freq)
160,report,10.9,1.564837,4.7,0.788136,0.02998,0.073906


In [80]:
## Lemmas more common in JP-EN than EN-EN
toks_jp = toks_df[(toks_df["JP-EN Mean Freq"] > toks_df["EN-EN Mean Freq"]) &
                 (toks_df["JP-EN Mean Count"] >= 1)]
toks_jp.sort_values("P (freq)", inplace=True)
results_tokjp = toks_jp[["Token", "JP-EN Mean Freq",
                         "EN-EN Mean Freq", "P (freq)"]].reset_index(drop=True)

results_tokjp.head(60)
#results_tokjp[:40].to_csv("savefiles/results_tokjp.csv")


,Token,JP-EN Mean Freq,EN-EN Mean Freq,P (freq)
0,detailed,0.170757,0.038129,0.005128
1,obtain,0.599597,0.195152,0.008790
2,common,0.586217,0.072855,0.011672
3,list,0.162510,0.017790,0.012019
4,thickness,0.184369,0.035410,0.013301
5,parametric,0.361579,0.036445,0.014723
6,respectively,0.832330,0.132969,0.019794
7,adopt,0.152203,0.010431,0.020350
8,activate,1.107028,0.226077,0.021462
9,study,6.115922,4.143915,0.024996


In [ ]:
## Lemmas less common in JP-EN than EN-EN
toks_en = toks_df[(toks_df["JP-EN Mean Freq"] < toks_df["EN-EN Mean Freq"]) &
                 (toks_df["EN-EN Mean Count"] >= 1)]
toks_en.sort_values("P (freq)", inplace=True)
results_token = toks_en[["Token", "JP-EN Mean Freq",
                         "EN-EN Mean Freq", "P (freq)"]].reset_index(drop=True)
results_token[:40].to_csv("savefiles/results_token.csv")

In [ ]:
# Sandbox

# new_words1, new_words2 = [], []
# words = ['gyrus', 'parietal-temporal-occipital', 'anxiety/depression']
# for word in words:
#     new_words += word.split('-')

# new_words

### For counting 1-grams

In [ ]:
def lexeme_counter(doc, string):
    
    """
    Function for getting raw lemma count in a document.
    """
        
    tokens = preprocess_tok(doc)
    counter = 0
    for tok in tokens:
        if str(tok) == str(string):
            counter += 1
    
    return counter

In [ ]:
results_df = initialize_results_df()

for lexeme in ["think", "consider", "report"]:
    
    new_name_n = "n_" + str(lexeme)
    new_name_adj = str(new_name_n) + "_adj"
    
    for i in range(0,20):
        # Pull text from df
        text = corpus_new.loc[i, "Text"]
        # Run scispaCy model
        doc = nlp(text)
        # Count lexeme (includes preprocessing)
        ct = lexeme_counter(doc, lexeme)
        # Put in results_df
        results_df.loc[i, new_name_n] = ct
        results_df.loc[i, new_name_adj] = (ct / results_df.loc[i, 'Word Count']) #* np.mean(results_df['Word Count'])

results_df

In [ ]:
compare_means('n_think')
compare_means('n_consider')
compare_means('n_think_adj')
compare_means('n_consider_adj')
compare_means('n_report')
compare_means('n_report_adj')

In [ ]:
for i in range(0,20):
    results_df.loc[i, 'Combined'] = results_df.loc[i, 'n_think'] + results_df.loc[i, 'n_consider']
    results_df.loc[i, 'Combined_adj'] = results_df.loc[i, 'Combined'] / results_df.loc[i, 'Word Count']
    
compare_means('Combined')
compare_means('Combined_adj')

### For 2,3-grams

In [ ]:
results_df = initialize_results_df()
cols = []

for lexeme in ["in this study", "therefore", "in addition", "bold response"]:
    
    new_name_n = "n_" + str(lexeme.replace(" ", ""))
    new_name_adj = str(new_name_n) + "_adj"
    cols.append(new_name_n)
    cols.append(new_name_adj)

    for i in range(0,20):
        text = corpus_new.loc[i, "Text"].lower()
        ct = text.count(lexeme)
        # Put in results_df
        results_df.loc[i, new_name_n] = ct
        results_df.loc[i, new_name_adj] = (ct / results_df.loc[i, 'Word Count']) #* np.mean(results_df['Word Count'])

results_df

In [ ]:
for col in cols:
    compare_means(col)

### Statistical testing
#### 1. Type/token ratio (lexical diversity)

In [ ]:
def type_token_ratio(doc):
    
    token_list = preprocess_tok(doc)
    n_type = len(unique(token_list))
    n_token = len(token_list)
    ttr = n_type/n_token
    
    return ttr

In [ ]:
def compare_means(var):
    
    from scipy.stats import ttest_ind
    
    jp_stats = list(results_df[results_df['Group'] == 'JP-EN'].loc[:, var])
    en_stats = list(results_df[results_df['Group'] == 'EN-EN'].loc[:, var])
    P = ttest_ind(jp_stats, en_stats).pvalue
    
    print(f'Mean {var}, JP-EN:  {np.mean(jp_stats)}')
    print(f'Mean {var}, EN-EN:  {np.mean(en_stats)}')
    print(f'Sig. (unpaired t-test): {P}')
    print('\n')

In [ ]:
# Calculate raw type/token ratio (lemmatized)

for i in range(0,20):
    # Pull text from df
    text = corpus_new.loc[i, "Text"]
    # Run scispaCy model
    doc = nlp(text)
    # Preprocess
    ttr = type_token_ratio(doc)
    # Put in results_df
    results_df.loc[i, 'Type/Token'] = ttr

results_df.head(5)

In [ ]:
# Calculate adjusted type/token ratio (lemmatized) divided by word count

for i in range(0,20):
    # Pull text from df
    text = corpus_new.loc[i, "Text"]
    # Split by word (" " for simplicity)
    word_ct = len(text.split(' '))
    # Put in results_df
    results_df.loc[i, 'Word Count'] = word_ct
    
results_df

In [ ]:
# Divide by mean word count of all documents
results_df['TTR_adj'] = (results_df['Type/Token'] / results_df['Word Count']) * np.mean(results_df['Word Count'])
    
results_df.head(6)

In [ ]:
# Compare mean stats between JP-EN and EN-EN corpora

compare_means('Type/Token')
compare_means('Word Count')
compare_means('TTR_adj')

Comments (4/10): H1 seems to be rejected. fMRI studies authored by Japanese scientists are just as lexically sophisticated as comparable docs authored by Anglophone counterparts. Perhaps this is a good thing: i.e., any differences discovered later are a product of linguistic features, rather than scientific knowledge/ignorance.

In [ ]:
# Examine the entities extracted by the mention detector. Note that they don't have types
# like in SpaCy, and they are more general (e.g including verbs) - these are any spans
# which might be an entity in UMLS, a large biomedical database.
print(a.ents)


#>>> (Myeloid derived suppressor cells,
#     MDSC,
#     immature,
#     myeloid cells,
#     immunosuppressive activity,
#     accumulate,
#     tumor-bearing mice,
#     humans,
#     cancer,
#     hepatocellular carcinoma,
#     HCC)

In [ ]:
# We can also visualise dependency parses
# (This renders automatically inside a jupyter notebook!):
from spacy import displacy
# displacy.render(next(doc2.sents), style='dep', jupyter=True)
displacy.render(a, style='dep', jupyter=True)

# See below for the generated SVG.
# Zoom your browser in a bit!